In [1]:
from google_play_scraper import search,app
from datetime import datetime
import pandas as pd
from concurrent.futures import *
from tqdm.notebook import tqdm

In [2]:
appids=[]
keywords=[]
df=pd.DataFrame(columns=['URL','Appid','App Title','Genre','Category','Release Time','Update Time','Score','Total Reviews'])

In [3]:
with open('Keywords.txt','r') as f:
    keywords=f.read().splitlines()

In [4]:
def leech_appids(keyword):
    global appids
    try:
        result=search(keyword)
        print(f'Keyword : {keyword} Total Result : {len(result)}')
        appids+=[item['appId'] for item in result]
    except Exception as e:
        if '429' in str(e):
            leech_appids(keyword)


with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(leech_appids,keywords), total=len(keywords)))


  0%|          | 0/153 [00:00<?, ?it/s]

Keyword : ai art Total Result : 30
Keyword : location application Total Result : 30
Keyword : weather information Total Result : 30
Keyword : artaist app Total Result : 30
Keyword : cryptocurrency app Total Result : 30
Keyword : sms activated Total Result : 20
Keyword : phone tracker Total Result : 30
Keyword : location finder Total Result : 30
Keyword : signature maker Total Result : 30
Keyword : image mixer Total Result : 30
Keyword : skills tutorial Total Result : 10
Keyword : cam translator Total Result : 8
Keyword : bank info Total Result : 11
Keyword : business empire Total Result : 5
Keyword : pdf forms Total Result : 30
Keyword : bluetooth ble Total Result : 30
Keyword : device codes Total Result : 10
Keyword : temporary sms Total Result : 10
Keyword : dual camera Total Result : 14
Keyword : mobile videos Total Result : 30
Keyword : features wifi Total Result : 30
Keyword : xray app Total Result : 12
Keyword : temperature scanner Total Result : 19
Keyword : truck simulator Tota

In [5]:
# clean duplicate Appid
print(f'Total Appids Before Cleaning Duplicate ==> {len(appids)}')
appids=set(appids)
print(f'Total Appids After Cleaning Duplicate ==> {len(appids)}')

Total Appids Before Cleaning Duplicate ==> 3988
Total Appids After Cleaning Duplicate ==> 1919


In [6]:
def leech_apps_detail(appid):
    global df
    try:
        app_detail=app(appid)
        print(f"Leeching Data with Appid ==> {appid}")
        try:
            r_time=datetime.strptime(app_detail['released'], '%b %d, %Y').strftime('%Y-%m-%d')
        except:
            r_time=''
        try:
            u_time=datetime.utcfromtimestamp(app_detail['updated']).strftime('%Y-%m-%d')
        except:
            u_time=''
        
        df.loc[len(df)]=[app_detail['url'],app_detail['appId'],app_detail['title'],app_detail['genre'],r_time,u_time,app_detail['score'],app_detail['reviews']]
    except Exception as e:
        if '429' in str(e):
            leech_apps_detail(appid)


with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(leech_apps_detail,appids), total=len(appids)))
    

  0%|          | 0/1919 [00:00<?, ?it/s]

Leeching Data with Appid ==> com.eaf.ethereum_angel_faucet
Leeching Data with Appid ==> com.rayo.savecurrentlocation
Leeching Data with Appid ==> com.yzxx.kclkbf
Leeching Data with Appid ==> dev.egl.com.intensidadbluetoothLeeching Data with Appid ==> com.mapswithme.maps.pro

Leeching Data with Appid ==> com.geozilla.family
Leeching Data with Appid ==> com.gotv.nflgamecenter.us.lite
Leeching Data with Appid ==> deezer.android.app
Leeching Data with Appid ==> com.gpsmapcamera.geotagginglocationonphoto
Leeching Data with Appid ==> co.windyapp.android
Leeching Data with Appid ==> com.stankomashstroi.industry
Leeching Data with Appid ==> phonetracker.locationtracker.gpstracker.familylocator
Leeching Data with Appid ==> com.mytruckingskills.v2
Leeching Data with Appid ==> com.infogama.codesverify
Leeching Data with Appid ==> com.sign.png
Leeching Data with Appid ==> com.plongo.nvcharts
Leeching Data with Appid ==> com.toralabs.deviceinfo
Leeching Data with Appid ==> com.jeyluta.timestampcame

In [7]:
df.head()

,URL,Appid,App Title,Genre,Category,Release Time,Update Time,Score,Total Reviews
0,https://play.google.com/store/apps/details?id=...,com.eaf.ethereum_angel_faucet,Ethereum Angel Faucet,Casual,Casual,2023-06-11,2023-09-01,4.500000,4
1,https://play.google.com/store/apps/details?id=...,com.rayo.savecurrentlocation,Save Location GPS,Maps & Navigation,Casual,2016-09-28,2023-09-26,4.710000,300
2,https://play.google.com/store/apps/details?id=...,com.yzxx.kclkbf,Truck Simulation,Puzzle,Casual,2023-08-09,2023-09-19,0.000000,0
3,https://play.google.com/store/apps/details?id=...,com.mapswithme.maps.pro,MAPS.ME: Offline maps GPS Nav,Travel & Local,Casual,2012-07-31,2023-09-18,4.136919,13133
4,https://play.google.com/store/apps/details?id=...,dev.egl.com.intensidadbluetooth,Bluetooth Signal Meter,Tools,Casual,2018-07-23,2022-07-09,3.970000,208


In [8]:
df.to_excel("app_detail.xlsx")